In [1]:
# ライブラリのインポート
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import os
import zipfile
import matplotlib.pyplot as plt
import japanize_matplotlib 
from matplotlib.ticker import MaxNLocator
from tqdm.auto import tqdm
from shapely.geometry import LineString, Point
import datetime as dt
from pathlib import Path
import pydeck as pdk
import numpy as np
import math
from geopy.distance import geodesic
import scipy
from shapely.wkt import loads

import gurobipy as gp
from gurobipy import Model, GRB
import logging
from itertools import product
import time


# データのインポート

In [2]:
PROJECT_ROOT = "../"
INTERMEDIATE_OUTPUT_DIR = PROJECT_ROOT + 'output/' + 'intermediate_data/'
# 入力ファイルパス
OPTIMIZATION_VARS_NPZ = INTERMEDIATE_OUTPUT_DIR + 'optimization_vars.npz'
FACILITY_NODE_PKL = INTERMEDIATE_OUTPUT_DIR + 'facility.pkl'
DESTINATION_NODE_PKL = INTERMEDIATE_OUTPUT_DIR + 'destination.pkl'



In [3]:
def load_data(vars_path: Path) -> dict:
    """最適化に必要な変数を.npzファイルから読み込む。"""
    logging.info(f"Loading optimization variables from {vars_path}...")
    try:
        # allow_pickle=True は、Numpy配列以外のオブジェクト（辞書など）を読み込むために必要
        data = np.load(vars_path, allow_pickle=True)
        # .item() を使って辞書オブジェクトを取り出す
        return {key: data[key].item() if data[key].dtype == 'O' else data[key] for key in data.files}
    except FileNotFoundError:
        logging.error(f"Error: Input file not found at {vars_path}")
        logging.error("Please run '01_prepare_data.py' first.")
        exit()


In [4]:
# データの読み込み
params = load_data(OPTIMIZATION_VARS_NPZ)
facility_df = pd.read_pickle(FACILITY_NODE_PKL)
destination_df = pd.read_pickle(DESTINATION_NODE_PKL)

# 最適化

In [11]:
# --- 最適化モデルクラス ---
class DroneDepotOptimizer:
    """ドローン配送拠点最適化モデルを構築・実行するクラス。"""

    def __init__(self, params: dict):
        """
        クラスの初期化。最適化に必要なパラメータを格納する。
        Args:
            params (dict): 01_prepare_data.pyで作成された変数の辞書
        """
        self.params = params
        self.model = None
        self.vars = {}

    def build(self, weights: dict):
        """
        Gurobiモデルを構築する。
        Args:
            weights (dict): 目的関数の重み {'profit': float, 'risk': float}
        """
        self.model = gp.Model("DroneDepotPlacement")

        # --- 変数定義 ---
        n_facilities = len(self.params['NODE_F'])
        n_destinations = len(self.params['NODE_D'])

        self.vars['open_f'] = self.model.addMVar(
            shape=n_facilities, vtype=GRB.BINARY, name="open_f"
        )
        self.vars['cover_d'] = self.model.addMVar(
            shape=n_destinations, vtype=GRB.BINARY, name="cover_d"
        )
        self.vars['connected_f_d'] = self.model.addMVar(
            shape=(n_facilities, n_destinations), vtype=GRB.BINARY, name="connected_f_d"
        )

        # --- 制約条件 ---
        # 1. 開設される施設数はK個
        self.model.addConstr(
            self.vars['open_f'].sum() == self.params['NUM_OPEN_FACILITIES'],
            name="num_open_facilities"
        )
        
        # 2. 閉鎖された施設からは接続できない
        # M: 十分に大きな数（ここでは需要地点の総数）
        M = n_destinations
        self.model.addConstr(
            self.vars['connected_f_d'].sum(axis=1) <= self.vars['open_f'] * M,
            name="no_connection_from_closed"
        )

        # 3. 各需要地点は、最大1つの施設からカバーされる
        #   sum(x_ij for i) = y_j
        self.model.addConstr(
            self.vars['connected_f_d'].sum(axis=0) == self.vars['cover_d'],
            name="at_most_one_connection"
        )

        # 4. ドローンの航続距離制約
        #   d_ij * x_ij <= E
        self.model.addConstr(
            self.vars['connected_f_d'] * self.params['DISTANCE_F_D'] <= self.params.get('UAV_ENDURANCE', 3),
            name="endurance_limit"
        )

        # --- 目的関数 ---
        # 目的1: 総収益性
        total_profit = (self.params['PROFITABILITY_F_D'] * self.vars['connected_f_d']).sum()
        
        # 目的2: カバーされた期待被害者数
        covered_risk = self.vars['cover_d'] @ (self.params['RISK_D'] * self.params['POPULATION_D'])
        
        # 正規化のための基準値（理想点）を取得
        b_profit = weights.get('b_profit', 1.0)
        b_risk = weights.get('b_risk', 1.0)

        print(b_profit, b_risk)  # デバッグ用ログ出力
        
        # # ゼロ割を避ける
        # if b_profit == 0: b_profit = 1.0
        # if b_risk == 0: b_risk = 1.0
        
        # 正規化された目的関数
        objective = (weights['profit'] * (total_profit / b_profit) +
                     weights['risk'] * (covered_risk / b_risk)) 
        
        self.model.setObjective(objective, GRB.MAXIMIZE)

    def solve(self) -> dict:
        """モデルを解き、結果を返す。"""
        self.model.optimize()
        
        if self.model.Status == GRB.OPTIMAL:
            logging.info(f"Optimal solution found. Objective value: {self.model.ObjVal:.4f}")
            # 結果を抽出
            results = {
                'status': 'optimal',
                'obj_val': self.model.ObjVal,
                'open_f': self.vars['open_f'].X.astype(int),
                'cover_d': self.vars['cover_d'].X.astype(int),
                'connected_f_d': self.vars['connected_f_d'].X.astype(int),
                'total_profit': (self.params['PROFITABILITY_F_D'] * self.vars['connected_f_d'].X).sum(),
                'covered_risk': self.vars['cover_d'].X @ (self.params['RISK_D'] * self.params['POPULATION_D'])
            }
            return results
        else:
            logging.warning(f"Optimization finished with status code: {self.model.Status}")
            if self.model.Status == GRB.INFEASIBLE:
                logging.info("Model is infeasible. Computing IIS to find the cause...")
                self.model.computeIIS()
                self.model.write("infeasible_model.ilp")
                logging.info("IIS written to 'infeasible_model.ilp'")
            return {'status': 'failed'}

In [12]:
# --- メイン実行関数 ---
def run_single_scenario(params: dict):
    """最適化の全フローを実行する。"""
    optimizer = DroneDepotOptimizer(params)
    
    # --- 1. 基準値（理想点）の計算 ---
    logging.info("--- Step 1: Calculating benchmark values (Utopia Point) ---")
    
    # 収益性のみ最大化
    logging.info("Maximizing for PROFIT only...")
    optimizer.build(weights={'profit': 1.0, 'risk': 0.0})
    profit_only_results = optimizer.solve()
    b_profit = profit_only_results.get('total_profit', 1.0)

    # リスク削減のみ最大化
    logging.info("Maximizing for RISK REDUCTION only...")
    optimizer.build(weights={'profit': 0.0, 'risk': 1.0})
    risk_only_results = optimizer.solve()
    b_risk = risk_only_results.get('covered_risk', 1.0)

    logging.info(f"Benchmark values: B_PROFIT = {b_profit:,.0f}, B_RISK = {b_risk:,.2f}")

    # --- 2. 多目的最適化（重み付け法） ---
    logging.info("--- Step 2: Running multi-objective optimization with weighted-sum ---")
    
    # 探索する重みのリスト
    omega_list = [1.0, 0.99, 0.95, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.01, 0.0]
    
    scenario_results = []

    for omega_profit in omega_list:
        omega_risk = round(1 - omega_profit, 2)
        logging.info(f"Solving for omega_profit={omega_profit:.2f}, omega_risk={omega_risk:.2f}...")
        
        weights = {
            'profit': omega_profit,
            'risk': omega_risk,
            'b_profit': b_profit,
            'b_risk': b_risk
        }
        
        optimizer.build(weights=weights)
        result = optimizer.solve()
        
        if result['status'] == 'optimal':
            result['omega_profit'] = omega_profit
            result['omega_risk'] = omega_risk
            scenario_results.append(result)

    return scenario_results

In [13]:
final_results = run_single_scenario(params)

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0x8864249c
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 280021 rows and 272096 columns
Presolve time: 0.10s
Presolved: 1031 rows, 8955 columns, 17901 nonzeros
Variable types: 0 continuous, 8955 integer (8955 binary)
Performing another presolve...
Presolve removed 9 rows and 9 columns
Presolve time: 0.04s

Root relaxation: objective 2.607846e+08, 8662 iterations, 0.12 seconds (0.21 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

# 結果の保存

In [14]:
FINAL_OUTPUT_DIR = PROJECT_ROOT + 'output/' + 'final_results/'
OPTIMIZATION_RESULTS_PKL = FINAL_OUTPUT_DIR + 'optimization_results.pkl'
OPTIMIZATION_PARAMS_PKL = FINAL_OUTPUT_DIR + 'optimization_params.npz'


In [15]:
def save_results(results: list, params: dict, facility_df: pd.DataFrame, destination_df: pd.DataFrame, result_path: str,params_path:str):
    """最適化結果を整形し、ファイルに保存する。"""
    logging.info(f"--- Step 3: Saving results and parameters to {result_path} and {params_path}---")
    
    if not results:
        logging.warning("No optimal results to save.")
        return

    # 結果をDataFrameに変換
    results_df = pd.DataFrame(results)
    
    # 未カバーのリスク（最小化したい対象）を計算
    max_risk = params['MAX_SUM_UNCOVERED_ISOLATED_PEOPLE']
    results_df['uncovered_risk'] = max_risk - results_df['covered_risk']

    # 保存用データ構造
    output_data = {
        'summary': results_df,
        'params': params,
        'facility_nodes': facility_df,
        'destination_nodes': destination_df
    }
    
    
    # 保存
    results_df.to_pickle(result_path)
    np.savez(params_path, **params)
    logging.info(f"Successfully saved optimization results to {result_path} and parameters to {params_path}")

In [16]:
save_results(final_results, params, facility_df, destination_df, OPTIMIZATION_RESULTS_PKL,OPTIMIZATION_PARAMS_PKL)

# Sensitivity Analysis

In [17]:
import time

In [18]:
# ## 感度分析のメインフロー
start_time = time.time()
base_params = params.copy()
all_results_collection = {}

In [19]:


# --- 分析1: 施設数 N_f の感度分析 ---
logging.info("--- Starting Sensitivity Analysis for Number of Facilities (N_f) ---")
n_f_list = [6, 8, 10, 12, 14, 16, 18, 20]
n_f_results = []
for n_f in tqdm(n_f_list, desc="N_f Analysis"):
    logging.info(f"--- Running for N_f = {n_f} ---")
    current_params = base_params.copy()
    current_params['NUM_OPEN_FACILITIES'] = n_f
    
    results_for_nf = run_single_scenario(current_params)
    for res in results_for_nf:
        res['n_f'] = n_f
    n_f_results.extend(results_for_nf)
    
all_results_collection['n_f_analysis'] = pd.DataFrame(n_f_results)
logging.info("--- Finished N_f analysis ---")

N_f Analysis:   0%|          | 0/8 [00:00<?, ?it/s]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0x784718ca
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 6e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 280021 rows and 272096 columns
Presolve time: 0.09s
Presolved: 1031 rows, 8955 columns, 17901 nonzeros
Variable types: 0 continuous, 8955 integer (8955 binary)
Performing another presolve...
Presolve removed 9 rows and 9 columns
Presolve time: 0.03s

Root relaxation: objective 2.148076e+08, 8155 iterations, 0.08 seconds (0.16 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

N_f Analysis:  12%|█▎        | 1/8 [01:04<07:34, 64.94s/it]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0x15112d43
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 280021 rows and 272096 columns
Presolve time: 0.11s
Presolved: 1031 rows, 8955 columns, 17901 nonzeros
Variable types: 0 continuous, 8955 integer (8955 binary)
Performing another presolve...
Presolve removed 9 rows and 9 columns
Presolve time: 0.04s

Root relaxation: objective 2.349327e+08, 8938 iterations, 0.14 seconds (0.25 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

N_f Analysis:  25%|██▌       | 2/8 [02:11<06:33, 65.61s/it]

...

N_f Analysis:  75%|███████▌  | 6/8 [06:41<02:15, 67.88s/it]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0xa0b6f5d4
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 280021 rows and 272096 columns
Presolve time: 0.08s
Presolved: 1031 rows, 8955 columns, 17901 nonzeros
Variable types: 0 continuous, 8955 integer (8955 binary)
Performing another presolve...
Presolve removed 9 rows and 9 columns
Presolve time: 0.03s

Root relaxation: objective 2.854229e+08, 7788 iterations, 0.14 seconds (0.23 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

N_f Analysis:  88%|████████▊ | 7/8 [07:50<01:07, 67.99s/it]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0x5ff5dacd
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 280021 rows and 272096 columns
Presolve time: 0.08s
Presolved: 1031 rows, 8955 columns, 17901 nonzeros
Variable types: 0 continuous, 8955 integer (8955 binary)
Performing another presolve...
Presolve removed 9 rows and 9 columns
Presolve time: 0.03s

Root relaxation: objective 2.886844e+08, 7601 iterations, 0.14 seconds (0.22 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

N_f Analysis: 100%|██████████| 8/8 [08:59<00:00, 67.41s/it]


In [20]:
# --- 分析2: ドローン航続距離 L_drone の感度分析 ---
logging.info("--- Starting Sensitivity Analysis for Drone Endurance (L_drone) ---")
l_drone_list = [1, 2, 3, 4]
l_drone_results = []
# 施設数はデフォルト値（12）に固定
base_params['NUM_OPEN_FACILITIES'] = 12 

for l_drone in tqdm(l_drone_list, desc="L_drone Analysis"):
    logging.info(f"--- Running for L_drone = {l_drone} km ---")
    current_params = base_params.copy()
    current_params['UAV_ENDURANCE'] = l_drone
    
    # 航続距離が変わると収益性も変わるため再計算
    ITEM_PRICE_MARGIN = 20 * 164.48
    FIX_COST = 1 * 164.48
    MAX_DELIVERY_COST = 1 * 164.48
    profit_per_person = ITEM_PRICE_MARGIN - FIX_COST - (MAX_DELIVERY_COST * current_params['DISTANCE_F_D'] / l_drone)
    current_params['PROFITABILITY_F_D'] = current_params['POPULATION_D'][np.newaxis, :] * profit_per_person

    results_for_ldrone = run_single_scenario(current_params)
    for res in results_for_ldrone:
        res['l_drone'] = l_drone
    l_drone_results.extend(results_for_ldrone)
    
all_results_collection['l_drone_analysis'] = pd.DataFrame(l_drone_results)
logging.info("--- Finished L_drone analysis ---")

end_time = time.time()
logging.info(f"Total execution time: {end_time - start_time:.2f} seconds.")

L_drone Analysis:   0%|          | 0/4 [00:00<?, ?it/s]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0xeb04bca1
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [6e-02, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 280253 rows and 279157 columns
Presolve time: 0.09s
Presolved: 799 rows, 1894 columns, 3608 nonzeros
Variable types: 0 continuous, 1894 integer (1894 binary)
Performing another presolve...
Presolve removed 414 rows and 857 columns
Presolve time: 0.02s

Root relaxation: objective 1.627296e+08, 600 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

L_drone Analysis:  25%|██▌       | 1/4 [01:03<03:09, 63.06s/it]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0xa256a23a
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+00, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 280053 rows and 276223 columns
Presolve time: 0.09s
Presolved: 999 rows, 4828 columns, 9632 nonzeros
Variable types: 0 continuous, 4828 integer (4828 binary)
Performing another presolve...
Presolve removed 19 rows and 23 columns
Presolve time: 0.02s

Root relaxation: objective 2.251558e+08, 3897 iterations, 0.03 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

L_drone Analysis:  50%|█████     | 2/4 [02:06<02:07, 63.53s/it]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0x8864249c
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 280021 rows and 272096 columns
Presolve time: 0.08s
Presolved: 1031 rows, 8955 columns, 17901 nonzeros
Variable types: 0 continuous, 8955 integer (8955 binary)
Performing another presolve...
Presolve removed 9 rows and 9 columns
Presolve time: 0.04s

Root relaxation: objective 2.607846e+08, 8662 iterations, 0.12 seconds (0.21 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

L_drone Analysis:  75%|███████▌  | 3/4 [03:14<01:05, 65.19s/it]

1.0 1.0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 22.5.0 22F66)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 281052 rows, 281051 columns and 841646 nonzeros
Model fingerprint: 0x8298ed83
Variable types: 0 continuous, 281051 integer (281051 binary)
Coefficient statistics:
  Matrix range     [3e-04, 3e+02]
  Objective range  [2e+03, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 280013 rows and 266710 columns
Presolve time: 0.08s
Presolved: 1039 rows, 14341 columns, 28677 nonzeros
Variable types: 0 continuous, 14341 integer (14341 binary)
Performing another presolve...
Presolve removed 5 rows and 5 columns
Presolve time: 0.06s

Root relaxation: objective 2.887756e+08, 13048 iterations, 0.39 seconds (0.74 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work


L_drone Analysis: 100%|██████████| 4/4 [04:31<00:00, 67.90s/it]


In [23]:
# ## 結果の保存
SENSITIVITY_RESULTS_PKL = FINAL_OUTPUT_DIR + 'sensitivity_analysis_results.pkl'
pd.to_pickle(all_results_collection, SENSITIVITY_RESULTS_PKL)
logging.info(f"Sensitivity analysis complete. Results saved to {SENSITIVITY_RESULTS_PKL}")

print("感度分析が完了しました。")

感度分析が完了しました。


NameError: name 'all_results_collection' is not defined